In [9]:
import pymysql
import pandas as pd
import pymysql.cursors
from tqdm import tqdm  # 진행률
import sys
import json
import mysqlConnecter as mc

In [5]:
# ✅ MySQL 연결 설정
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    database='FinancialStatement',  # 데이터베이스 이름
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

In [2]:
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with conn.cursor() as curser:

        # 1. 데이터 베이스 목록 보기
        curser.execute("SHOW DATABASES")
        databases = curser.fetchall()
        print('사용 가능한 데이터베이스 목록:')
        for db in databases:
            print("-", db['Database'])

        # 2. 데이터 베이스 선택하기
        db_name = '회원관리DB'
        db_exists = any(db['Database'] == db_name for db in databases)

        if db_exists:
            print('\n데이터 베이스 선택합니다')
            curser.execute(f'USE {db_name}')
        else:
            exit()

        
        # 3. 현재 데이터베이스 테이블 목록 가져오기
        curser.execute('SHOW TABLES')
        tables = curser.fetchall()
        print('\n사용가능한 테이블 목록')
        for table in tables:
            print(f'- {table}')


        # 4. 데이터프레임으로 전체 조회
        sql = "SELECT * FROM 회원정보"
        curser.execute(sql)
        result = curser.fetchall()

        df = pd.DataFrame(result)
        display(df)
finally:
    conn.close()

사용 가능한 데이터베이스 목록:
- FinancialStatement
- information_schema
- mysql
- performance_schema
- sys
- 회원관리DB

데이터 베이스 선택합니다

사용가능한 테이블 목록
- {'Tables_in_회원관리db': '회원정보'}


,id,이름,나이,이메일
0,1,홍길동,25,hong@example.com
1,2,송빌리,30,billy@naver.com


In [ ]:
# 손익계산서, 대차대조표, 현금흐름표 로드
def save_income_statement_to_db(stockInfo):
    try:
        # 1️⃣ FinancialStatement에서 데이터 가져오기
        df = stockInfo.stock.financials  # 손익계산서 가져오기
        df = df.where(pd.notna(df), None)  # NaN 값을 None으로 변환
        
        # 2️⃣ 데이터가 없는 경우 예외 처리
        if df.empty:
            print(f"❌ 손익계산서 데이터 없음")
            return

        df = df.T  # 연도를 행으로 변환
        df.reset_index(inplace=True)  # 인덱스 초기화
        df.columns = ['year'] + [col.replace(' ', '_').replace('-', '_') for col in df.columns[1:]]
        
        # 3️⃣ symbol 및 year 컬럼 추가 및 변환
        df['symbol'] = symbol
        df['year'] = df['year'].dt.year  # 연도를 정수형으로 변환

        # 4️⃣ 현재 MySQL 테이블의 컬럼 목록 가져오기
        with conn.cursor() as cursor:
            cursor.execute("""
                SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS 
                WHERE TABLE_NAME = 'IncomeStatements'
            """)
            existing_columns = {row["COLUMN_NAME"] for row in cursor.fetchall()}  # 컬럼명 집합

        # 5️⃣ 누락된 컬럼 확인 및 추가
        new_columns = set(df.columns) - existing_columns - {"symbol", "year"}  # 기존 컬럼 제외
        if new_columns:
            with conn.cursor() as cursor:
                for column in new_columns:
                    alter_sql = f"ALTER TABLE IncomeStatements ADD COLUMN `{column}` DOUBLE NULL;"
                    cursor.execute(alter_sql)
                conn.commit()
            print(f"✅ 새 컬럼 추가됨: {', '.join(new_columns)}")

        # 6️⃣ MySQL에 저장할 데이터 정리
        db_columns = ['symbol', 'year'] + [col for col in df.columns if col not in ['symbol', 'year']]
        df = df[db_columns]  # 컬럼 순서 맞추기

        # 7️⃣ MySQL에 데이터 저장
        with conn.cursor() as cursor:
            sql = f"""
            INSERT INTO IncomeStatements ({', '.join(db_columns)})
            VALUES ({', '.join(['%s'] * len(db_columns))})
            ON DUPLICATE KEY UPDATE 
            {', '.join([f"{col} = VALUES({col})" for col in db_columns[2:]])};
            """
            data = [tuple(x) for x in df.values]
            cursor.executemany(sql, data)
            conn.commit()
        
        # print(f"✅ {symbol} - 데이터 저장 완료")

    except Exception as e:
        print(f"⚠️ {symbol} - Income Statement 오류 발생: {e}")


def save_balance_sheet_to_db(stockInfo):
    try:
        # 1️⃣ FinancialStatement에서 데이터 가져오기
        df = stockInfo.stock.balance_sheet  # 대차대조표 가져오기

        # 2️⃣ 데이터가 없는 경우 예외 처리
        if df.empty:
            print(f"❌ 대차대조표 데이터 없음")
            return

        df = df.T  # 연도를 행으로 변환
        df.reset_index(inplace=True)

        # 3️⃣ NaN 값을 None으로 변환하여 MySQL에서 오류 방지
        df = df.where(pd.notna(df), None)

        # 4️⃣ 컬럼 이름 변환 (MySQL에서 허용되지 않는 공백/특수문자 → `_` 변경)
        df.columns = ['year'] + [col.replace(' ', '_').replace('-', '_') for col in df.columns[1:]]

        # ✅ 기존에 symbol 컬럼이 있으면 제거 (중복 방지)
        df = df.drop(columns=['symbol'], errors='ignore')

        # ✅ symbol 컬럼 추가
        df['symbol'] = symbol
        df['year'] = df['year'].dt.year  # 연도를 정수형으로 변환

        # ✅ MySQL 테이블 컬럼 리스트 확인
        with conn.cursor() as cursor:
            cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BalanceSheet'")
            existing_columns = {row['COLUMN_NAME'] for row in cursor.fetchall()}

        # ✅ 추가해야 할 컬럼 찾기
        missing_columns = set(df.columns) - existing_columns
        if missing_columns:
            with conn.cursor() as cursor:
                for col in missing_columns:
                    sql = f"ALTER TABLE BalanceSheet ADD COLUMN `{col}` DECIMAL(15,2) NULL;"
                    cursor.execute(sql)
                conn.commit()
            print(f"🛠️ {symbol} - 누락된 컬럼 추가 완료: {missing_columns}")

        # ✅ 컬럼 순서 정렬
        column_order = ['symbol', 'year'] + [col for col in df.columns if col not in ['symbol', 'year']]
        df = df[column_order]  

        # 5️⃣ `ON DUPLICATE KEY UPDATE`를 사용하여 덮어쓰기
        with conn.cursor() as cursor:
            sql = f"""
            INSERT INTO BalanceSheet ({', '.join(column_order)})
            VALUES ({', '.join(['%s'] * len(column_order))})
            ON DUPLICATE KEY UPDATE 
            {', '.join([f"{col} = VALUES({col})" for col in column_order[2:]])};
            """
            data = [tuple(x) for x in df.values]
            cursor.executemany(sql, data)
            conn.commit()

        # print(f"✅ {symbol} - 데이터 저장 완료 (컬럼 추가 + 덮어쓰기 지원)")

    except Exception as e:
        print(f"⚠️ {symbol} - Balance Sheet 오류 발생: {e}")


def save_cash_flow_to_db(stockInfo):
    try:
        # 1️⃣ FinancialStatement에서 데이터 가져오기
        df = stockInfo.stock.cash_flow  # 현금흐름표 가져오기

        # 2️⃣ 데이터가 없는 경우 예외 처리
        if df.empty:
            print(f"❌ 현금흐름표 데이터 없음")
            return

        df = df.T  # 연도를 행으로 변환
        df.reset_index(inplace=True)

        # 3️⃣ NaN 값을 None으로 변환하여 MySQL에서 오류 방지
        df = df.where(pd.notna(df), None)

        # 4️⃣ 컬럼 이름 변환 (MySQL에서 허용되지 않는 공백/특수문자 → `_` 변경)
        df.columns = ['year'] + [col.replace(' ', '_').replace('-', '_') for col in df.columns[1:]]

        # ✅ 기존에 symbol 컬럼이 있으면 제거 (중복 방지)
        df = df.drop(columns=['symbol'], errors='ignore')

        # ✅ symbol 컬럼 추가
        df['symbol'] = symbol
        df['year'] = df['year'].dt.year  # 연도를 정수형으로 변환

        # ✅ MySQL 테이블 컬럼 리스트 확인
        with conn.cursor() as cursor:
            cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'CashFlow'")
            existing_columns = {row['COLUMN_NAME'] for row in cursor.fetchall()}

        # ✅ 추가해야 할 컬럼 찾기
        missing_columns = set(df.columns) - existing_columns
        if missing_columns:
            with conn.cursor() as cursor:
                for col in missing_columns:
                    sql = f"ALTER TABLE CashFlow ADD COLUMN `{col}` DECIMAL(15,2) NULL;"
                    cursor.execute(sql)
                conn.commit()
            print(f"🛠️ {symbol} - 누락된 컬럼 추가 완료: {missing_columns}")

        # ✅ 컬럼 순서 정렬
        column_order = ['symbol', 'year'] + [col for col in df.columns if col not in ['symbol', 'year']]
        df = df[column_order]  

        # 5️⃣ `ON DUPLICATE KEY UPDATE`를 사용하여 덮어쓰기
        with conn.cursor() as cursor:
            sql = f"""
            INSERT INTO CashFlow ({', '.join(column_order)})
            VALUES ({', '.join(['%s'] * len(column_order))})
            ON DUPLICATE KEY UPDATE 
            {', '.join([f"{col} = VALUES({col})" for col in column_order[2:]])};
            """
            data = [tuple(x) for x in df.values]
            cursor.executemany(sql, data)
            conn.commit()

        # print(f"✅ {symbol} - 데이터 저장 완료 (컬럼 추가 + 덮어쓰기 지원)")

    except Exception as e:
        print(f"⚠️ {symbol} - Cash Flow 오류 발생: {e}")


def save_company_info_to_db(stockInfo):
    try:
        info = stockInfo.stock.info
        symbol = stockInfo.ticker.upper()  # 주식 심볼 가져오기

        # ✅ MySQL 테이블 컬럼 리스트 확인
        with conn.cursor() as cursor:
            cursor.execute("SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Company'")
            existing_columns = {row['COLUMN_NAME']: row['DATA_TYPE'] for row in cursor.fetchall()}

        # ✅ MySQL 컬럼과 매칭되는 데이터 추출
        data = {key: info.get(key, None) for key in info.keys()}
        data.pop("companyOfficers", None)  # ❌ companyOfficers 제거
        data["symbol"] = symbol  # 심볼 추가

        # ✅ 추가해야 할 컬럼 찾기 (데이터 타입 구분)
        missing_columns = set(data.keys()) - set(existing_columns.keys())

        if missing_columns:
            with conn.cursor() as cursor:
                for col in missing_columns:
                    value = data[col]

                    # 숫자형 데이터는 FLOAT, BIGINT 등으로 저장
                    if isinstance(value, int):
                        col_type = "BIGINT NULL"
                    elif isinstance(value, float):
                        col_type = "FLOAT NULL"
                    elif isinstance(value, str) and len(value) > 255:
                        col_type = "TEXT NULL"  # 255자를 초과하는 문자열은 TEXT
                    else:
                        col_type = "VARCHAR(255) NULL"  # 기본적으로 VARCHAR(255)

                    sql = f"ALTER TABLE Company ADD COLUMN `{col}` {col_type};"
                    cursor.execute(sql)
                conn.commit()
            print(f"🛠️ {symbol} - 누락된 컬럼 추가 완료: {missing_columns}")


        # ✅ 데이터 변환 (dict, list → JSON 문자열)
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                data[key] = json.dumps(value)  # JSON 문자열로 변환
                
        # ✅ 데이터 삽입 또는 업데이트
        columns = ", ".join(data.keys())
        placeholders = ", ".join(["%s"] * len(data))
        update_clause = ", ".join([f"{key} = VALUES({key})" for key in data.keys() if key != "symbol"])

        with conn.cursor() as cursor:
            sql = f"""
            INSERT INTO Company ({columns})
            VALUES ({placeholders})
            ON DUPLICATE KEY UPDATE {update_clause};
            """
            cursor.execute(sql, tuple(data.values()))
            conn.commit()
        print(f"✅ {symbol} - 데이터 저장 완료")

    except Exception as e:
        print(f"⚠️ {symbol} - 오류 발생: {e}")

symbols = []

try:
    with conn.cursor() as cursor:
        cursor.execute("SELECT symbol FROM Company")
        result = cursor.fetchall()

        symbols =  [row['symbol'] for row in result]

except Exception as e:
    print(f"⚠️ 오류 발생: {e}")

finally:
    cursor.close()


stockInfo = fs.FinancialStatement()

# ✅ 기업 리스트에 대해 CashFlowStatement 저장 실행
for symbol in tqdm(symbols[1:], desc="Processing Companies"):

    stockInfo.setCompany(symbol)

    if stockInfo.stock:
        save_company_info_to_db(stockInfo)
        # save_income_statement_to_db(stockInfo)
        # save_balance_sheet_to_db(stockInfo)
        # save_cash_flow_to_db(stockInfo)
    else:
        print(f"❌ {symbol} 데이터 없음")


# ✅ MySQL 연결 종료
conn.close()

In [ ]:

#--------------------------
# NYSE 데이터 정보 모두 DB에 저장
#--------------------------


# ✅ MySQL 연결 설정
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    database='FinancialStatement',  # 데이터베이스 이름
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

# ✅ Excel 파일 불러오기
df = pd.read_excel('nyse_symbol.xlsx')

# ✅ 필요 없는 URL 컬럼 제거
df = df[['Symbol', 'Name']]  # URL 컬럼 제외

print(df.loc[3672])

# ✅ NaN 값이 있으면 None으로 변환
df = df.where(pd.notna(df), None)

# ✅ Symbol 값이 NaN 또는 빈 문자열("")인 행 찾기
invalid_indices = df[df['Symbol'].isna() | (df['Symbol'].astype(str).str.strip() == "")].index.tolist()


if invalid_indices == None or len(invalid_indices) > 0:
    print(f"symbol 값이 비어 있습니다 : {invalid_indices}")
    sys.exit(1)


# ✅ 데이터 삽입 또는 업데이트 처리
with conn.cursor() as cursor:
    sql = """
    INSERT INTO Company (symbol, name)
    VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE 
    name = VALUES(name);
    """
    data = [tuple(x) for x in df.values]  # 데이터 변환

    # ✅ 데이터를 한 번에 삽입하는 것이 아니라, 일부씩 나눠서 삽입
    batch_size = 100  # 100개씩 실행
    for i in range(0, len(data), batch_size):
        try:
            cursor.executemany(sql, data[i:i+batch_size])
            conn.commit()  # ✅ 여기서 멈춘다면, 트랜잭션 충돌 가능성 있음
            print(f"✅ {i + batch_size}개 데이터 처리 완료!")
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            conn.rollback()  # ✅ 오류 발생 시 롤백
            break  # ✅ 반복문 중단 후 오류 확인

print("✅ 데이터 삽입 완료 (URL 컬럼 제외, 중복 시 덮어쓰기 처리됨)")

# ✅ MySQL 연결 종료
conn.close()

In [ ]:

import commonHelper as ch
import yFinanceDownloader as yf

def save_fs_to_db(stockInfo, type, dateType):
    
    infoName = ch.getStrFinancialStatementType(type)
    dateName = ch.getStrFinancialStatementType(dateType)

    tableName = f"{infoName}_{dateName}"

    try:
        df = stockInfo.getFsData(type, dateType)
        df = df.where(pd.notna(df), None)

        if df.empty:
            print(f"❌ {tableName} 데이터 없음")
            return
        
        symbol = stockInfo.stock.info.get('symbol','N/A')
        name = stockInfo.stock.info.get('shortName','N/A')

        df = df.T
        df.reset_index(inplace=True)
        df.rename(columns={"index": "date"}, inplace=True)
        df.columns = [col.replace(' ', '_').replace('-', '_') for col in df.columns]
        df['symbol'] = symbol
        df['name'] = name

        with conn.cursor() as cursor:
            cursor.execute(f"""
                SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS
                WHERE TABLE_NAME = '{tableName}'
            """)

            existing_columns = {row['COLUMN_NAME'] for row in cursor.fetchall()}
            
        new_columns = set(df.columns) - existing_columns - {"symbol", "year"}
        if new_columns:
            with conn.cursor() as cursor:
                for col in new_columns:
                    value = df[col].iloc[0]

                    # 날짜 타입 확인 (문자열이지만 날짜 변환 가능한 경우)
                    if isinstance(value, str):  
                        try:
                            pd.to_datetime(value)  # 변환 가능 여부 체크
                            col_type = "DATE NULL"  # 변환 가능하면 DATE로 설정
                        except ValueError:
                            col_type = "TEXT NULL" if len(value) > 128 else "VARCHAR(128) NULL"
                    
                    # Pandas datetime 타입 (직접 변환)
                    elif isinstance(value, pd.Timestamp):
                        col_type = "DATE NULL"

                    # 숫자형 데이터는 FLOAT, BIGINT 등으로 저장
                    elif isinstance(value, int):
                        col_type = "BIGINT NULL"
                    elif isinstance(value, float):
                        col_type = "FLOAT NULL"
                    
                    # 기본적으로 VARCHAR(128)으로 설정
                    else:
                        col_type = "VARCHAR(128) NULL"  

                    sql = f"ALTER TABLE {tableName} ADD COLUMN `{col}` {col_type};"

                    cursor.execute(sql)
                
                conn.commit()
            print(f"🛠️ {symbol} - 누락된 컬럼 추가 완료: {new_columns}")

        with conn.cursor() as cursor:
            columns = ", ".join([f"`{col}`" for col in df.columns])
            placeholders = ", ".join(["%s"] * len(df.columns))
            update_clause = ", ".join([f"`{col}` = VALUES(`{col}`)" for col in df.columns if col not in ["date", "symbol"]])

            insert_sql = f"""
                INSERT INTO {tableName} ({columns})
                VALUES ({placeholders})
                ON DUPLICATE KEY UPDATE {update_clause};
            """

            data = [tuple(row) for row in df.itertuples(index=False, name=None)]
            cursor.executemany(insert_sql, data)
            conn.commit()

        print(f"✅ {symbol} - {tableName} 데이터 삽입 완료")

    except Exception as e:
        print(f"⚠️ {symbol} - {tableName} 오류 발생: {e}")


stockInfo = fs.FinancialStatement()
mysqlConn = mc.MySQLConnector()
mysqlConn.connect()

symbols = mysqlConn.getSymbolList()

for symbol in tqdm(symbols[:1], desc="Processing Companies"):
    
    stockInfo.setCompany(symbol)

    if stockInfo.stock:
        save_fs_to_db(stockInfo, FSType.INCOME_STATEMENT, FSDateType.QUARTER)
        save_fs_to_db(stockInfo, FSType.BALANCE_SHEET, FSDateType.QUARTER)
        save_fs_to_db(stockInfo, FSType.CASH_FLOW, FSDateType.QUARTER)
    else:
        print(f"❌ {symbol} 데이터 없음")


mysqlConn.disconnect()

In [ ]:
def insert_stock_data(df):
    """
    fetch_stock_data로 가져온 DataFrame을 MySQL StockPrices 테이블에 저장
    :param df: fetch_stock_data로 가져온 데이터프레임
    :param db_config: MySQL 접속 정보 (host, user, password, db, port)
    """

    # ✅ MySQL 연결 설정
    conn = pymysql.connect(
        host='localhost',
        user='root',
        password='1234',
        database='Stock',  # 데이터베이스 이름
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )

    # DB 연결
    connection = conn

    # DataFrame을 튜플 리스트로 변환 (adj_close는 Close로 채워넣음)
    data_to_insert = list(
        df[['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close']].itertuples(index=False, name=None)
    )

    sql = """
        INSERT INTO StockPrices (date, symbol, open, high, low, close, volume, adj_close)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            open = VALUES(open),
            high = VALUES(high),
            low = VALUES(low),
            close = VALUES(close),
            volume = VALUES(volume),
            adj_close = VALUES(adj_close);
    """

    try:
        with connection.cursor() as cursor:
            cursor.executemany(sql, data_to_insert)
            print(f"{cursor.rowcount} rows inserted/updated successfully.")
        connection.commit()
    except Exception as e:
        print("Error occurred:", e)
        connection.rollback()
    finally:
        connection.close()



[*********************100%***********************]  1 of 1 completed


Price,Date,Symbol,Open,High,Low,Close,Volume
0,2025-03-11,AAPL,223.809998,225.839996,217.449997,220.839996,76087700
1,2025-03-12,AAPL,220.179993,221.750000,219.270004,220.134995,11631991


1 rows inserted/updated successfully.


In [ ]:

data = fs.FinancialStatement.getStockData(['AAPL'],'2025-03-11')
display(data)
insert_stock_data(data)